In [2]:
import pandas as pd

In [3]:
class Environment:
    """
    차트 데이터는 우선 간단하게 종가와 거래량만 제공한다.
    """
    
    PRICE_IDX = 0  # 종가의 위치
    VOLUME_IDX = 1 # 거래량의 위치
    
    
    def __init__(self, chart_data=None, balance=0, transaction=0.05):
        self.chart_data = chart_data
        self.idx = 0
        self.state = chart_data.iloc[self.idx]
        self.next_state = chart_data.iloc[self.idx+1]
        self.balance = [0]
        self.current_price = self.chart_data.iloc[self.idx, 0]
        self.next_price = self.chart_data.iloc[self.idx+1, 0]
        
    def reset(self):
        self.idx = 0
        state = self.chart_data.iloc[self.idx]
        return state
    
    def step(self, action):
        if (self.chart_data).shape[0] > self.idx+1:
            print(self.idx)
            s_prime = self.chart_data.iloc[self.idx+1]
            reward = self.get_reward(action)
            done = False
            self.idx += 1
            return (s_prime, reward, done)
        return (None,None,True)
    
    def get_reward(self, action):
        """
        action은 다음 스텝의 행동을 의미한다. 
        왜냐하면, 현재까지의 데이터를 기반으로 action을 선택해야하기 때문이다. 
        (input data가 close price, volume인데 action을 하는 시점에서는 input data를 얻지 못함)
        """
        
        """     
        손실에 더 큰 penalty를 줌으로써, 손실 회피형 Agent를 구성
        근데 이러면, 장기적인 보상에 대해서는 감지를 잘 못하지 않나...?
        
            => GAE가 이 의문을 해결해준다고 생각함. n-step TD를 통해서 미래의 보상을 고려할 수 있음. 
            
        """        
        self.current_price = self.chart_data.iloc[self.idx, 0]
        self.next_price = self.chart_data.iloc[self.idx+1, 0]
        print(self.current_price, self.next_price)
        
        if action == 1:
            if self.next_price >= self.current_price:
                reward = 1 * (self.next_price - self.current_price)
            else:
                reward = 1.2 * (self.next_price - self.current_price)        
        elif action == -1:
            if self.next_price >= self.current_price:
                reward = 1.2 * (self.next_price - self.current_price) 
            else:
                reward = 1 * (self.next_price - self.current_price) 
        return reward
        

In [4]:
df = pd.read_csv('../upbit_data/202308_KRW-BTC_ohlcv.csv', index_col=0)
df = df[['close', 'volume']]

In [5]:
env=  Environment(df)

In [6]:
s = env.reset()

In [170]:
env.step(1)

163
37783000.0 37791000.0


(close     3.779100e+07
 volume    2.545718e+00
 Name: 2023-08-01 11:44:00, dtype: float64,
 8000.0,
 False)

In [372]:
df['mid_price']

,close,volume
index,,
2023-08-01 09:00:00,37922000.0,5.338512
2023-08-01 09:01:00,37924000.0,2.175930
2023-08-01 09:02:00,37923000.0,0.893822
2023-08-01 09:03:00,37916000.0,2.212888
2023-08-01 09:04:00,37915000.0,1.822514
...,...,...
2023-08-31 23:55:00,36977000.0,2.138066
2023-08-31 23:56:00,36980000.0,1.803252
2023-08-31 23:57:00,36980000.0,2.107548
